## Introduction
Concurrency means performing two or more activities at the same time, which are conceptually different. There are 2 ways computers can perform concurrency
* Hardware concurrency : Modern computers have multiple processors(multiple chips/cores), these processors can perform different activities at the same time even within the same program. These are known as harware threads, one hardware thread <=> 1 processor core.
* Software concurrency : Modern operating systems support software threading. A multi threaded program can perform multiple activities at the same time, these thread activities are managed by the operating system. Typically there are more software threads than harware threads. 

## Motivation
Some of the important motivations for concurrency are to have high server throughput to support millions of connections simultaneously, to have a responsive UI(separation of concerns), to do fast numeric computations(in parallel) for games/multimedia and effective use of multi-core hardware.

## Threads
Each thread is an independent execution environment, has its own execution stack and processor state. Threads are like light weight processes, less overhead to start, smaller task switching overhead and easy to share data between threads. Non-threaded C++ programs have a single thread, main is the entry point function for this thread. Multi-threaded C++ program also has the main thread, main thread starts addition threads, these inturn can start further threads. Each thread has its own entry point function. When the entry point function returns the thread will exit. Modern C++ added support for concurrency, standard library was updated to add features to create and manage threads and all standard library classes and functions were made thread safe. The basic unit of concurrency in C++ is the std::thread class. std::thread constructor takes a callable object which is the threads entry point function. The entry point function cannot be overloaded and any return value from this function is ignored. The constructor itself will start the execution of the thread.
```
//Callable object, thread entry point
void hello()
{
    std::cout << "Hello, thread!" << std::endl;
}

std::thread thr(hello);

//Blocking call, will wait for the thread to complete
thr.join();
```
By default the destructor of the std::thread class will call std::terminate function. If the thread has exited then std::terminate will not be called. The entry point function can be any callable object, functor class, lambda expression, member function pointer or object returned by std::bind/std::function. We can pass arguments to the thread function, as discussed the first argument to the std::thread class is the callable object, any argument after that is interpreted as an argument to the entry point function.
```
void hello(std::string str) 
{
    std::cout << str << std::endl;
}

std::thread thr(hello, "Hello, thread!");

thr.join();
```
std::thread object owns the arguments, so we can pass lvalue passed by value or rvalue passed by move.
```
void hello(std::string &&str) 
{
    std::cout << str << std::endl;
}

std::string str{"Hello, thread!"}
std::thread thr(hello, std::move(str));

thr.join();
```
We can also pass the argument by reference using std::ref. std::ref will return a reference_wrapper object, this object will have a reference to its argument as a member. So reference_wrapper looks like a value object, but it is actually a reference.
```
void hello(std::string &str) 
{
    str = "xyz";
}

std::string str{"Hello, thread!"}
std::thread thr(hello, std::ref(str));

thr.join();

std::cout << str << std::endl;
```
Member function as the threads entry point.
```
class greeter
{
public:
    void hello()
    {
        std::cout << "Hello, member function thread!" << std::endl;
    }
};

greeter greet;

//Pass the pointer to the member function and the pointer to the object
std::thread thr(&greeter::hello, &greet);
thr.join();
```
Lambda expression as the threads entry point.
```
int i = 3;

std::thread thr([&i]
                { i *= 2;});
thr.join();

std::cout << i << std::endl;
```

## Processors
* Single processor : The program runs on a single processor which reads/writes instructions/data to/from RAM. Multi-threaded programs on a single processor will appear to run concurrently due to time slicing, where each thread runs on the CPU for a short time and then another thread will get a chance to run and so on. This is all controlled by the operating system thread scheduler, this is similar to the scheduler that the operating system uses for scheduling different running processes. This way the thread scheduler will do pre-emptive task switching. Time slicing requires context switching to be done every time the scheduler moves to the next thread. This means the processor will have to stop processing the instructions during the context switch.
* Single processor and cache : CPU is much faster and spents a lot of time waiting for the RAM to respond. To address this cache memory was introduced. CPU reads/writes from/to the cache, cache is responsible to sync with main memory(RAM). Cache is physically close to the CPU, much smaller than main memory, it is made with static RAM which is faster than dynamic RAM used by main memory. Therefore communicating with cache is much faster than main memory. Cache is managed by the cache controller.
* Multiple processors : Modern processors have multiple cores and hyperthreads. Cores are multiple processors within the same silicon chip. Hyperthreads are duplicate circuitry within a core enough to run a seperate thread, with its own execution stack. Now there are several levels of cache, level 1 cache is inside the core and private to the core, level 2 cache is either private or shared by neibouring cores and very close to the cores, level 3 cache is common and shared by all the cores on the chip. Cache controller cordinates between the cache's and maintians data coherency. Modern processors have additional optimizations like pre-fetcher and store buffer. Pre-fetcher, is where next executing instructions and data are fetched in groups into the cache to improve fetch time. Store buffer, is a buffer between the core and the level 1 cache, processor will copy into the store buffer and continue, cache controller will copy from store buffer to cache level 1.

## Synchronization issues
Different threads executing on different cores may share data, this can cause synchronization issues. If a thread on a particular core updates the shared data, it will take some cycles for the value to propagate through the store buffer's, cache's, main memory to the cache of another core which has a thread that is using the same data. If this second thread uses this data during this time, it is using invalid data. With synconization techniques like mutexes and atomic variables we have to make sure that updated data is available to the second core before it uses the shared data.